In [88]:
from dataclasses import dataclass
from math import lcm
import itertools

def pad(x, n):
    return [None for i in range(n - len(x))] + x

@dataclass
class Problem():

    def __init__(self, init_dict):
        self.legend = init_dict['legend']
        self.cons = init_dict['cons']
        self.hidden_cons = []
        self.poss = init_dict['poss']
        self.solved = init_dict['solved']
        self.error = ''
        self.assumptions = []
    
    @classmethod
    def from_crypt(cls, string):
        split = string.split(" ")
        str_1, str_2, str_3 = split[0], split[2], split[4]
        carry = ["C"+str(i) for i in range(1, len(str_3))]
        legend_base = list(set(str_1 + str_2 + str_3))
        legend = legend_base + carry
        str1 = pad([legend.index(i) for i in str_1], len(str_3))
        str2 = pad([legend.index(i) for i in str_2], len(str_3))
        str3 = [legend.index(i) for i in str_3]
        carry = [legend.index(i) for i in carry] + [None]
        cons = []
        print(str1, str2, str3, carry, legend)
        for i in range(len(str3)):
            x = [0 for i in range(len(legend)+ 1)]
            if str1[i] != None:
                x[str1[i]] += 1
            if str2[i] != None:
                x[str2[i]] += 1
            if str3[i] != None:
                x[str3[i]] -= 1
            if carry[i] != None:
                x[carry[i]] += 1
            if i > 0:
                x[carry[i-1]] -= 10
            cons.append(x)
        
        heads = list(dict.fromkeys([legend.index(i) for i in [str_1[0], str_2[0], str_3[0]]]))
        poss = [[j for j in range(10)] for i in range(len(legend_base))] + [[0,1] for i in range(len(carry) - 1)]
        for i in heads:
            poss[i].remove(0)

        return cls({'legend': legend, 'cons': cons, 'poss': poss, 'solved': [None for i in range(len(legend))]})

    def __repr__(self):
        return f'Problem({self.legend}, {self.cons}, {self.poss}, {self.solved})'
    
    def solve(self, i, x):
        if i not in range(len(self.legend)) or x not in range(10):
            return False
        elif x in self.solved[:i] + self.solved[i+1:]:
            return False
        elif self.solved[i] != None and self.solved[i] != x:
            return False
        self.solved[i] = x
        for j in range(len(self.cons)):
            if self.cons[j][i] != 0:
                self.cons[j][-1] += x*self.cons[j][i]
                self.cons[j][i] = 0
        for j in range(len(self.poss)):
            if x in self.poss[j]:
                self.poss[j].remove(x)
        self.poss[i] = [x]
        return True
    
    def assume(self, i, x):
        self.assumptions.append((i, x))
        return self.solve(i, x)



    def check_for_solved(self):
        x = []
        for i in range(len(self.legend)):
            if len(self.poss[i]) == 1 and self.solved[i] == None:
                self.solve(i, self.poss[i][0])
                print(f'Solved {self.legend[i]} to {self.poss[i][0]}')
                x.append((i, self.poss[i][0]))
        return x
    
    def check_for_impossible(self):
        for i in range(len(self.legend)):
            if len(self.poss[i]) == 0:
                return True
        return False
    
    def check_constraints(self):
        for constraint in self.cons:
            unsolved = [i for i in range(len(constraint) - 1) if (constraint[i] != 0 and self.solved[i] == None)]
            if len(unsolved) == 0:
                inputs = [constraint[i]*self.solved[i] for i in range(len(constraint) - 1) if constraint[i] != 0]
                if sum(inputs) != -1*constraint[-1]:
                    return False
        return True

    
    def solve_simple_cons(self):
        newly_solved = []
        for i in range(len(self.cons)):
            x = [k for k, v in enumerate(self.cons[i][:-1]) if v != 0]

            if len(x) == 1 and self.solved[x[0]] == None:
                print(f'Solved {self.legend[x[0]]} to {-1*int(self.cons[i][-1]/self.cons[i][x[0]])}')
                self.solve(x[0], -1*int(self.cons[i][-1]/self.cons[i][x[0]]))

                newly_solved.append((x[0], self.cons[i][-1]))
        return newly_solved


    def basic_solve(self):
        x = self.check_for_solved() + self.solve_simple_cons()
        while len(x) > 0:
            x = self.check_for_solved() + self.solve_simple_cons()
        return self.check_for_impossible()


        
    #Take two constraints and unify them given a variable
    def unify_cons(self, i, j, x):
        if i not in range(len(self.cons)) or j not in range(len(self.cons)) or x not in range(len(self.legend) - 1):
            return False
        elif self.cons[i][x] == 0 or self.cons[j][x] == 0:
            return False
        else: 
            lcm = lcm(self.cons[i][x], self.cons[j][x])
            new1 = [self.cons[i][k] * lcm/self.cons[i][x] for k in range(len(self.cons[i]))]
            new2 = [self.cons[j][k] * lcm/self.cons[j][x] for k in range(len(self.cons[j]))]
            new = [new1[k] - new2[k] for k in range(len(new1))]
            self.cons.append(new)
            return True
            
    # Take a constraint and a variable and restruct possible values for that variable
    def find_poss(self, i, x):
        if i not in range(len(self.cons)) or x not in range(len(self.legend)):
            self.error = 'Invalid constraint or variable'
            return False
        elif self.cons[i][x] == 0:
            self.error = 'Variable not in constraint'
            return False
        else:
            other_variables = [k for k in range(len(self.cons[i])-1) if k != x and self.cons[i][k] != 0]
            print(other_variables)
            coeff = [self.cons[i][k] for k in other_variables]
            integer = self.cons[i][-1]
            z = self.cons[i][x]
            if len(other_variables) == 0:
                x_value = -1*int(self.cons[i][-1]/self.cons[i][x[0]])
                if x_value not in self.poss[x]:
                    self.error = 'No possible value for variable'
                    return False
                else:
                    self.solve(i, x_value)
                    return True
            else:
                poss = []
                for perm in itertools.product(*[self.poss[k] for k in other_variables]):
                    y = -1*int((integer + sum([coeff[k]*perm[k] for k in range(len(coeff))])))
                    if y % z == 0:
                        poss.append(int(y/z))
                if len(poss) == 0:
                    self.poss[x] = []
                    # Add error message
                    self.error = 'No possible value for variable'
                    return False
                else:
                    self.poss[x] = list(set(poss))
                    return True

    def show_state(self):
        result = 'Constraints:\n'
        for constraint in self.cons:
            for i in range(len(constraint) - 1):
                if constraint[i] > 0:
                    result += f'+ {constraint[i]}{self.legend[i]} '
                elif constraint[i] < 0:
                    result += f'- {-1*constraint[i]}{self.legend[i]} '
            result += f'= {constraint[-1]}'
            result += '\n'
        result += 'Possibilities:\n'
        for i in range(len(self.legend)):
            result += f'{self.legend[i]}: {self.poss[i]}\n'
        result += 'Solved:\n'
        for i in range(len(self.legend)):
            if self.solved[i] != None:
                result += f'{self.legend[i]}: {self.solved[i]}\n'
        result += 'Assumptions:\n'
        for i in self.assumptions:
            result += f'{self.legend[i[0]]}: {i[1]}\n'
        return result






In [89]:
x = Problem.from_crypt('I + I = U')

[1] [1] [0] [None] ['U', 'I']


In [95]:
print(x.show_state())

Constraints:
= 0
Possibilities:
U: [2]
I: [1]
Solved:
U: 2
I: 1
Assumptions:
U: 2



In [94]:
x.basic_solve()

Solved I to 1


False

In [92]:
x.assume(0, 2)

True

In [53]:
print(x.solve(0, 4))

True


In [43]:
x.check_constraints()

False

In [47]:
x.solved

[4, None]

In [78]:
x.legend

['U', 'I']

In [86]:
x.find_poss(0, 1)

[0]


True

In [59]:
x.error

'Invalid constraint or variable'

In [87]:
x.poss

[[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4]]